# Notebook for testing data for pilot 3

Pilot 3 added more checks for engagement early in the experiment.
Iteration names:
'pre-pilot3':

## 1. Read in packages and set up server connection
This first section will read in necessary packages for anaysis and establish file paths and connections to the mongo db server

In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import drawing_utils as drawing
import importlib
import scoring

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional-abstractions']
coll = db['two-towers']

# which iteration name should we use?
iterationName = 'pilot4'

## look up number of trials (including paired-practice)
numTrials = 13

In [ ]:
# look into failType

## 2. Construct tidy dataframe with game data

### establish connection to mongo
First thing you need to do is to establish an ssh tunnel (aka remote port forwarding) to the server, so that requests to the mongodb can be made "as if" the mongodb server is running on your local computer. Run this from the command line before you begin data analysis if you plan to fetch data from mongo:

`ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org`

#### Pass in list of iterations, returns list of total games and completed games for each

In [4]:
## get list of all gameIDs in database
#iterationList = ['livetest0','pilot0','pilot2']

iterationList = ['pilot4','pilot4b']

for iteration in iterationList:
    #get total games
    total_games = coll.find({'iterationName':iteration}).distinct('gameid')
    print('There are {} total games in iteration: {}.'.format(len(total_games), iteration))

    ## get list of complete gameIDs
    gameIDs = coll.find({'iterationName':iteration}).distinct('gameid')
    complete_games = [g for g in gameIDs if len(coll.find({'gameid':g}).distinct('trialNum')) == numTrials]
    print('There are {} complete games in iteration: {}.'.format(len(complete_games), iteration))

There are 28 total games in iteration: pilot4.
There are 15 complete games in iteration: pilot4.
There are 75 total games in iteration: pilot4b.
There are 34 complete games in iteration: pilot4b.


In [5]:
broken_games = []
complete_games = [g for g in complete_games if g not in broken_games]

def construct_tidy_dataframe(eventType = 'chatMessage', 
                             complete_games = [],
                             iterationName = 'pilot3',
                             remove_workerID = True):
    '''
    input: list of complete games and name of event Type
    '''
    event2name = {'chatMessage':'chat', 'block':'block', 'endTrial':'trial', 'exitSurvey':'exit'}
    L = pd.DataFrame()
    for g, this_gameID in enumerate(complete_games):
        print('Analyzing game {} | {} of {}'.format(this_gameID, g+1, len(complete_games)))
        clear_output(wait=True) 

        ### extract records 
        #loop over iteration names??
        X = coll.find({ '$and': [{'iterationName':iterationName}, 
#                                  {"$or":[{'iterationName':'Exp2Pilot3'},
#                                  {'iterationName':'Exp2Pilot3_batch2'}]}
                                 {'gameid': this_gameID}, {'eventType': eventType}]}).sort('time') 
        
        li = list(X)        
        _L = pd.DataFrame(li)  

        ## concat with previous game's dataframe
        if L.shape[0]==0:
            L = _L
        else: 
            L = pd.concat([L,_L], axis=0)     

    ## postprocessing
    if remove_workerID and 'workerId' in L.columns:
        L = L.drop('workerId',axis=1)
        
    if eventType in ['block','endTrial','chatMessage']:
        L['practice'] = L.trialNum == 'practice'
        L.trialNum = pd.to_numeric(L['trialNum'], errors='coerce')

    ## save out group dataframe to csv dir
    out_path = os.path.join(csv_dir,'compabs_{}_{}.csv'.format(event2name[eventType],iterationName))
    print('Saving dataframe out to CSV dir at path: {}'.format(out_path))    
    L.to_csv(out_path)             

    return L


In [6]:
## construct dataframe for each datatype
dataTypes = coll.distinct('eventType')
for thisDataType in dataTypes:
    X = construct_tidy_dataframe(eventType=thisDataType, complete_games=complete_games, iterationName=iterationName)


AttributeError: 'DataFrame' object has no attribute 'trialNum'

## 3. Create a full DF from DB
We can read in a full df from the db and do some basic checks to make sure the data look right

#### check outcome for specific workerID
Must pass in workerID and corresponding iteration

In [8]:
f = pd.DataFrame(coll.find({'iterationName': 'pilot3'}))
f['gameid'].unique()

array(['5809-2a5e096b-a425-4bc7-b495-cf74cb60ded6',
       '6598-80d65299-0654-43b4-9f27-08572427cb2d'], dtype=object)

In [9]:
a = pd.DataFrame(coll.find({'failType': 'instructionsQuiz'}))
a

,_id,workerId,failType,iterationName
0,5f47c827df57f9222c29cc11,A21WWDLXGJQHWZ,instructionsQuiz,NaN
1,5f47c83fdf57f9222c29cc12,A2UUZ3WDQBQ21A,instructionsQuiz,NaN
2,5f47d778df57f9222c29cc22,A3ENDV5PRQI2U2,instructionsQuiz,NaN
3,5f47d871df57f9222c29cc29,AQ8B4BC1EUWN,instructionsQuiz,NaN
4,5f47def1df57f9222c29ccbb,A2K4X1QPAZ0DZX,instructionsQuiz,NaN
...,...,...,...,...
151,5f5922a31034572e66881a99,A22UMPYYQBXT0F,instructionsQuiz,pilot4b
152,5f5922ce1034572e66881aaa,A2X89NXIKI5Y6,instructionsQuiz,pilot4b
153,5f5923ea1034572e66881b02,A3BX6V29MSZ6F0,instructionsQuiz,pilot4b
154,5f5926511034572e66881bc2,AUR5KDE75HSZ9,instructionsQuiz,pilot4b


In [10]:
b = pd.DataFrame(coll.find({'failType': 'soloPractice'}))
b

,_id,workerId,failType,iterationName
0,5f4d42293c9d171947610dac,undefined,soloPractice,pilot3_debug
1,5f4fe6481034572e6687fa48,A1EP0JITQIOYIQ,soloPractice,pilot4
2,5f5154821034572e6687fead,A3HYLVFUDKJFKC,soloPractice,pilot4
3,5f51817a1034572e6688038b,A18XVBWB2A270I,soloPractice,pilot4
4,5f5804551034572e6688068d,A1222QAH015PT5,soloPractice,pilot4b
5,5f5814e01034572e66880c33,A3KUF6W5ANCT85,soloPractice,pilot4b
6,5f5816741034572e66880c6b,A307GIMK5CZLVY,soloPractice,pilot4b
7,5f5916651034572e66881604,A3FF5CCILJAWYT,soloPractice,pilot4b
8,5f591a6b1034572e668817a3,A2ZGAH6NOSU2PO,soloPractice,pilot4b


In [11]:
c = pd.DataFrame(coll.find())

c[c.pairedPracticeFails==3]

,_id,iterationName,gameid,time,workerId,assignmentId,leftTarget,rightTarget,trialNum,turnNum,...,strategy,role,totalLength,cumulativeScore,cumulativeBonus,trialScore,workerID,individualPracticeAttempts,failType,pairedPracticeFails
6754,5f4d40de3c9d171947610dab,pilot3_debug,1068-ffe5a500-3e57-41ef-ac07-79b48edfcab3,1.598898e+12,,,vertical,horizontal,NaN,0.0,...,aervwav,listener,30657.0,NaN,NaN,NaN,NaN,None,NaN,3.0
7154,5f4fea141034572e6687fb09,pilot4,1879-aef64278-6e02-4755-907a-075dd2383d24,1.599073e+12,A2WUXZOY4CRF5R,3E47SOBEYUW78Q4UOK6BV9E7C80ICM,vertical,horizontal,NaN,0.0,...,"As the words are limited, I tried to use simpl...",speaker,1028320.0,NaN,NaN,NaN,NaN,1,NaN,3.0
7165,5f4fea3e1034572e6687fb14,pilot4,1879-aef64278-6e02-4755-907a-075dd2383d24,1.599073e+12,A2837NCV9OXBFZ,3LO69W1SU7DMF3S5B9Z7DGRER4MLGD,vertical,horizontal,NaN,0.0,...,I do not use strategy,listener,636853.0,NaN,NaN,NaN,NaN,0,NaN,3.0
8080,5f5152b81034572e6687fea7,pilot4,9536-a5ff2384-044a-4905-bcf8-794c7cc3a926,1.599165e+12,A2VO8C41JJIQY9,3RGU30DZTE8GCN0RJRLPA6YJJCBJML,vertical,horizontal,NaN,0.0,...,I was trying to give as much as information po...,speaker,417145.0,NaN,NaN,NaN,NaN,1,NaN,3.0
8082,5f5153611034572e6687fea9,pilot4,9536-a5ff2384-044a-4905-bcf8-794c7cc3a926,1.599165e+12,ARYL3C6N9SVV1,36DSNE9QZ9Y4CQL3W7022ANOECQOJ5,vertical,horizontal,NaN,0.0,...,My partner provided very poor instructions~~~ ...,listener,541208.0,NaN,NaN,NaN,NaN,0,NaN,3.0
8152,5f5158391034572e6687feef,pilot4,5312-bde29e51-6dd2-4407-999b-b1628c52f363,1.599167e+12,AKX9W0NF71OI7,3ERET4BTVQ9WGNNFC3HW6ZSCY379KG,horizontal,vertical,NaN,0.0,...,I tried to think about how I would need instru...,speaker,390555.0,NaN,NaN,NaN,NaN,0,NaN,3.0
8160,5f5158611034572e6687fef7,pilot4,5312-bde29e51-6dd2-4407-999b-b1628c52f363,1.599167e+12,AIZUOHKQT14OM,3E47SOBEYUW78Q4UOK6BV9E7CJICIK,horizontal,vertical,NaN,0.0,...,I look the box and place the blocks~~~,listener,427938.0,NaN,NaN,NaN,NaN,2,NaN,3.0
8508,5f515de31034572e66880053,pilot4,1323-16b3e9d9-f8f6-4829-a2a9-a97ea5e3dfe0,1.599168e+12,A2EOOF9D135HQ1,3DIP6YHAPGSTGICVBP0F88KWUZQE87,horizontal,vertical,NaN,0.0,...,"Mine would have been to follow directions, but...",listener,899052.0,NaN,NaN,NaN,NaN,0,NaN,3.0
8539,5f515e2c1034572e66880072,pilot4,1323-16b3e9d9-f8f6-4829-a2a9-a97ea5e3dfe0,1.599168e+12,A1P9HVNYM5POH,3SLE99ER0RDATB1YAOY2KKBFQCSZB7,horizontal,vertical,NaN,0.0,...,i done my task by intuitive,speaker,986330.0,NaN,NaN,NaN,NaN,1,NaN,3.0
9312,5f517c571034572e66880377,pilot4,5133-a539d2c0-e73e-4ed4-8c64-a2141d8a8528,1.599176e+12,A32LHQYJY0ELFS,388U7OUMFB107XCX2R70E0DXQRYR06,horizontal,vertical,NaN,0.0,...,it's encouraging,listener,428581.0,NaN,NaN,NaN,NaN,1,NaN,3.0


## 4. Trial Level Data
We're going to construct a spearate df for each event type (chat, block, endTrial, Survey) which will make analysis a little easier.
We also want to be able to concatenate multiple iterations for grouped analysis of previous versions.

### Create DF for each event


In [14]:
event_types = ['block', 'chat','exit','trial']
iteration_list = ['pilot3']
df_dict = {}
for event in event_types:
    event_dict = {}
    df_name = 'df_'+ event
    for iteration in iteration_list:
        df_temp = 'df_'+event+"_"+iteration # the name for the dataframe
        file_name = '../results/csv/compabs_{}_{}.csv'.format(event,iteration)
        event_dict[df_temp] = pd.read_csv(file_name)
    df_dict[df_name] = pd.concat(event_dict.values(), ignore_index = True)

In [18]:
df_block = df_dict['df_block']
df_chat = df_dict['df_chat']
df_exit = df_dict['df_exit']
df_trial = df_dict['df_trial']

##### Light data cleaning

In [19]:
#create columns for char and word counts
df_chat['word_count'] = df_chat['content'].str.split(' ').str.len()
df_chat['char_count'] = df_chat['content'].str.len()
df_chat["timeElapsedInTurn"] = pd.to_numeric(df_chat['timeElapsedInTurn'])

In [20]:
df_block['w'] = df_block['width']
df_block['h'] = df_block['height']
df_block["timeElapsedInTurn"] = pd.to_numeric(df_block['timeElapsedInTurn'])

#### Write to data frames

In [21]:
#read in dataframes from each eventType
df_block.to_csv('df_block.csv', header=True, index=False) 
df_chat.to_csv('df_chat.csv', header=True, index=False) 
df_exit.to_csv('df_exit.csv', header=True, index=False) 
df_trial.to_csv('df_trial.csv', header=True, index=False) 